In [1]:
library(jsonlite)
library(dsl)

dsl v0.1.0 successfully loaded. See ?dsl for help. Note this is an early alpha release and backwards compatability may not be maintained.



# Introduction

Let:
* $N$ be the total number of samples in the dataset
* $n$ be the number of samples selected for expert annotation
* $M_{true}$ be a logistic regression trained with expert annotations for all $N$ samples
* $M_{sub}$ be a logistic regression trained with expert annotations on only the subset of $n$ samples selected for expert annotation
* $M_{dsl}$ be a logistic regression trained with DSL using predicted annotations for all $N$ samples and expert annotations for a subset of $n$ samples

As part of some experiments to try to measure the effective sample size of different datasets when using DSL (along the lines of [this paper](https://osf.io/preprints/socarxiv/j3bnt_v3)), we tried to compare the performance of $M_{sub}$ and $M_{dsl}$ when varying $n$. To this end, we measured the RMSE of the coefficients of both models compared to the coefficients of $M_{true}$. We expected that $M_{dsl}$ would always have a smaller RMSE than $M_{sub}$ and that in both cases the RMSE would go to 0 as $\frac{n}{N} \to 1$.

However, for one of of the datasets we tested we observed a different trend: the RMSE for $M_{dsl}$ became larger than for $M_{sub}$ as we increased $n$ and it never converged to zero:

![rmse.pdf](rmse.png)

Our question is: **why is this happening?** So far we haven't been able to figure it out. Interestingly, these effects do not manifest in our other datasets (I included a second dataset based on Amazon reviews for which DSL behaves as expected).

Below we provide an example of this phenomenon for the extreme case where $n = N = 10000$, where we expect that $M_{sub} = M_{dsl} = M_{true}$

# The Data

The data is a small subset of the [misinfo-general dataset](https://huggingface.co/datasets/ioverho/misinfo-general) of newspaper articles. We trimmed the dataset down to articles from two sources: The Guardian and The Sun. The goal of the logistic regression is to predict whether the article comes from The Guardian or The Sun based on certain independent variables of the text, such as the text length in words, the length of the title, and so forth. We balanced the dataset to have 5000 samples for each class. Predictions for the outcome label were obtained by training a classifier on the DistilBERT embeddings of the articles and their true labels.

In [2]:
data <- fromJSON("misinfo.json")
head(data)

x1   x2  x3  x4  y y_hat
1 1595 235  98 106 0 0    
2 1562 237  59  78 0 1    
3 2574 410  69  95 0 0    
4 1828 289  37 102 0 0    
5 1995 323  88 118 0 0    
6 1852 300 115 119 0 0

In [3]:
cat("Number of samples:", nrow(data), "\n")
cat("Class balance:", sum(data$y) / nrow(data), "\n")

Number of samples: 10000 
Class balance: 0.5 


# Training

In [4]:
M_true <- glm(
    y ~ x1 + x2 + x3 + x4,
    data = data,
    family = binomial
)
summary(M_true)


Call:
glm(formula = y ~ x1 + x2 + x3 + x4, family = binomial, data = data)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  3.9042463  0.1455957   26.82   <2e-16 ***
x1           0.0074069  0.0002452   30.21   <2e-16 ***
x2          -0.0411265  0.0015480  -26.57   <2e-16 ***
x3          -0.0264629  0.0010629  -24.90   <2e-16 ***
x4          -0.0653850  0.0017329  -37.73   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 13862.9  on 9999  degrees of freedom
Residual deviance:  7047.1  on 9995  degrees of freedom
AIC: 7057.1

Number of Fisher Scoring iterations: 6


In [5]:
M_dsl <- dsl(
    model = "logit",
    formula = y ~ x1 + x2 + x3 + x4,
    predicted_var = "y",
    prediction = "y_hat",
    data = data,
    seed = 0
)
summary(M_dsl)

Cross-Fitting: 1/10..2/10..3/10..4/10..5/10..6/10..7/10..8/10..9/10..10/10..==================
DSL Specification:
Model:  logit
Call:  y ~ x1 + x2 + x3 + x4

Predicted Variables:  y
Prediction:  y_hat

Number of Labeled Observations:  10000
Random Sampling for Labeling with Equal Probability: Yes

Coefficients:
            Estimate Std. Error CI Lower CI Upper p value    
(Intercept)   4.3469     0.1804   3.9933   4.7006  0.0000 ***
x1            0.0005     0.0002   0.0001   0.0010  0.0100   *
x2            0.0034     0.0016   0.0003   0.0065  0.0163   *
x3           -0.0229     0.0013  -0.0254  -0.0204  0.0000 ***
x4           -0.0715     0.0023  -0.0761  -0.0670  0.0000 ***
---
Signif. codes: 0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
95% confidence intervals (CI) are reported.

# Results

Since we actually have true labels for all samples when doing DSL, we expect the coefficients of both logistic regressions to be very similar, and therefore we expect both the standardised bias and the RMSE to be close to 0. However, we do not observe this:

In [6]:
true_coeffs = M_true$coefficients
dsl_coeffs = M_dsl$coefficients
print(dsl_coeffs)
print(true_coeffs)

  (Intercept)            x1            x2            x3            x4 
 4.3469481141  0.0005172022  0.0033685361 -0.0228986108 -0.0715496354 
 (Intercept)           x1           x2           x3           x4 
 3.904246329  0.007406862 -0.041126492 -0.026462945 -0.065385028 


In [7]:
cat("RMSE:", sqrt(mean((dsl_coeffs - true_coeffs)^2)), "\n")

RMSE: 0.1990291 
